<a href="https://colab.research.google.com/github/degartHub/nocountry-h12-25-equipo27-datascience/blob/main/H12_25_L_Equipo_27_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook para el proyecto de predicción de atrasos de vuelos - HACKATHON ONE

## Data Engineer (DE)

Sección para las tareas de Data Engineer.

Encargado: Ismael Cerda

### Selección y Limpieza de Datos

Base de datos obtenida de: https://www.kaggle.com/datasets/jimschacko/airlines-dataset-to-predict-a-delay?select=Airlines.csv

In [ ]:
import pandas as pd

url="https://raw.githubusercontent.com/degartHub/nocountry-h12-25-equipo27-datascience/refs/heads/main/data/Airlines.csv"
df = pd.read_csv(url)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           539383 non-null  int64 
 1   Airline      539383 non-null  object
 2   Flight       539383 non-null  int64 
 3   AirportFrom  539383 non-null  object
 4   AirportTo    539383 non-null  object
 5   DayOfWeek    539383 non-null  int64 
 6   Time         539383 non-null  int64 
 7   Length       539383 non-null  int64 
 8   Delay        539383 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 37.0+ MB


In [ ]:
df["Time"].agg(["min", "max"])

,Time
min,10
max,1439


La base de datos cuenta con un total de 539.383 registros y un total de 9 columnas, siendo estas:

- <u>**id**</u>= Identifica la fila del registro.

- <u>**Airline**</u>= Aerolínea.

- <u>**Flight**</u>= Número de la aeronave.

- <u>**Airport From**</u>= Aeropuerto de salida.

- <u>**Airport To**</u>= Aeropuerto de destino.

- <u>**DayOfWeek**</u>= Día de la semana (en números).

- <u>**Time**</u>= Hora de salida medida en minutos a partir de la medianoche (rango de [10,1439], lo que podría ser el equivalente a un día).

- <u>**Lenght**</u>= Duración del vuelo en minutos.

- <u>**Delay**</u>= Con retraso (1), sin retraso (0).

In [ ]:
df.sample(n=5)

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
60716,60717,WN,20,DAL,LBB,6,740,65,1
208350,208351,AS,258,SEA,LAX,1,480,155,1
81220,81221,MQ,2789,DFW,SGF,7,1065,75,1
446065,446066,DL,109,ATL,MCO,7,1085,94,1
47164,47165,9E,3868,DTW,TVC,5,930,68,1


Las colummnas a eliminar serán:
- ID: Es un identificador para la tabla en sí
- Flight: Identifica el número de avión, no es relevante.

In [ ]:
df = df.drop(columns=["id", "Flight"])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Airline      539383 non-null  object
 1   AirportFrom  539383 non-null  object
 2   AirportTo    539383 non-null  object
 3   DayOfWeek    539383 non-null  int64 
 4   Time         539383 non-null  int64 
 5   Length       539383 non-null  int64 
 6   Delay        539383 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 28.8+ MB


## Feature Architect (FA)

Sección para las tareas de Feature Architect

Encargado: Eduardo Ayala

### Ingeniería de Atributos

**Acerca de los datos faltantes**

Nos faltan fechas y distancia recorrida en km

*   Si observamos los valores mínimos y máximos de la columna `Time` se ve que están en el rango (10, 1439), que corresponde aproximadamente a los minutos que tiene un día completo, y, aparte, hay una columna `Lenght` con el tiempo de vuelo. Como son ~540 mil vuelos no se puede asumir que son todos del mismo día, es algo más parecido a los vuelos de 1 mes en USA. Así que se crearán las fechas de partida de manera sintética para 1 mes.
*   Adicionalmente a lo anterior, nos piden la distancia en kilómetros, pero tenemos el tiempo de vuelo; así que haremos la conversión estimando $800\frac{km}{\text{hr}}$ (velocidad promedio de un vuelo comercial).

Sumado a lo anterior, vamos a renombrar las columnas de acuerdo al contrato con backend.

**Acerca de la eficiencia y reducción de memoria**

A continuación se describen las optimizaciones aplicadas enfocadas en reducir el uso de memoria y mantener consistencia con backend.

1. Transformación de fechas a formato datetime

   - Las columnas de fecha se almacenaron como `datetime64[ns]` en lugar de strings.
   - La columna **`fecha_partida`** se generó en formato **ISO-8601** (`YYYY-MM-DDTHH:MM:SS`), compatible con backend.

 2. Creación de columnas de hora y día de la semana
    - Las variables temporales (`hora_salida`, `hora_llegada`, `dia_semana`) se extrajeron desde columnas datetime.
    - Se guardaron como **int8**: ocupan 1 byte por celda y tienen rango acotado (horas y días), sin pérdida de información.

3. One-Hot-Encoding de variables categóricas

    - Se utilizó `OneHotEncoder(handle_unknown='ignore')` para evitar errores en producción.
    - El resultado del encoding se convirtió a **uint8**.

4. Escalado de la variable de distancia (cuando el modelo lo requiere)

    - La variable **`distancia_km`** se convirtió a **float32**.
    - Para modelos que lo requieren (ej. Regresión Logística). Se usó `StandardScaler(with_mean=False)`, compatible con matrices sparse, evitando densificar el dataset.

In [ ]:
# -----------------------------------------------------
# Paso 1: Crear columnas de hora y día de la semana
# Eliminar columnas innecesarias
# -----------------------------------------------------

import numpy as np
import pandas as pd

np.random.seed(42)
VELOCIDAD_PROMEDIO_KMH = 800

# -----------------------------------------------------
# RENOMBRAR SEGÚN CONTRATO BACKEND
# -----------------------------------------------------
df = df.rename(columns={
    'Airline': 'aerolinea',
    'AirportFrom': 'origen',
    'AirportTo': 'destino',
    'Length': 'duration_min',
    'Delay': 'retraso'  # solo para entrenamiento
})

# -----------------------------------------------------
# CALCULAR DISTANCIA EN KM
# -----------------------------------------------------
df['distancia_km'] = (df['duration_min'] / 60) * VELOCIDAD_PROMEDIO_KMH

# -----------------------------------------------------
# FECHAS BASE (DICIEMBRE 2018)
# -----------------------------------------------------
start_date = pd.to_datetime('2018-12-01')
end_date = pd.to_datetime('2018-12-31')
random_days = np.random.randint(0, (end_date - start_date).days + 1, size=len(df))
df['FlightDate'] = (start_date + pd.to_timedelta(random_days, unit='D')).normalize()

# -----------------------------------------------------
# FECHA/HORA DE SALIDA
# -----------------------------------------------------
df['DepartureDateTime'] = df['FlightDate'] + pd.to_timedelta(df['Time'], unit='m')

# -----------------------------------------------------
# FECHA PARTIDA FORMATO ISO-8601 (CONTRATO BACKEND)
# -----------------------------------------------------
df['fecha_partida'] = df['DepartureDateTime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# -----------------------------------------------------
# ELIMINAR COLUMNAS QUE YA NO SE USAN
# -----------------------------------------------------
df = df.drop(columns=['duration_min'])

# -----------------------------------------------------
# VERIFICACIÓN
# -----------------------------------------------------
print("Columnas finales después de la Parte 1 (incluye 'retraso' solo para entrenamiento):")
print(df.head())

Columnas finales después de la Parte 1 (incluye 'retraso' solo para entrenamiento):
  aerolinea origen destino  DayOfWeek  Time  retraso  distancia_km FlightDate  \
0        CO    SFO     IAH          3    15        1   2733.333333 2018-12-07   
1        US    PHX     CLT          3    15        1   2960.000000 2018-12-20   
2        AA    LAX     DFW          3    20        1   2200.000000 2018-12-29   
3        AA    SFO     DFW          3    20        1   2600.000000 2018-12-15   
4        AS    ANC     SEA          3    30        0   2693.333333 2018-12-11   

    DepartureDateTime        fecha_partida  
0 2018-12-07 00:15:00  2018-12-07T00:15:00  
1 2018-12-20 00:15:00  2018-12-20T00:15:00  
2 2018-12-29 00:20:00  2018-12-29T00:20:00  
3 2018-12-15 00:20:00  2018-12-15T00:20:00  
4 2018-12-11 00:30:00  2018-12-11T00:30:00  


In [ ]:
# -----------------------------------------------------
# PARTE 2: CREAR COLUMNAS DE HORA Y DÍA DE LA SEMANA
# -----------------------------------------------------
# ADVERTENCIA: Las fechas son inventadas. El 'dia_semana' se conserva
# desde la columna original 'DayOfWeek' y no necesariamente coincide con la nueva fecha.
# La columna 'retraso' se mantiene solo para entrenamiento interno.

# Convertir 'fecha_partida' a datetime
df['fecha_partida_dt'] = pd.to_datetime(df['fecha_partida'])

# Crear columna de hora de salida como objeto time (HH:MM)
df['hora_salida'] = df['fecha_partida_dt'].dt.time

# Conservar día de la semana original desde 'DayOfWeek'
df['dia_semana'] = df['DayOfWeek'].astype('int8')  # del dataset original

# Reducir memoria: distancia y retraso
df['distancia_km'] = df['distancia_km'].astype('float32')
df['retraso'] = df['retraso'].astype('uint8')  # binario

# Eliminar columnas temporales redundantes
#if 'Time' in df.columns:                                                             NS: No eliminamos la columna Time, ya que mide la hora de salida en numeros enteros
#    df = df.drop(columns=['Time'])
df = df.drop(columns=['fecha_partida_dt'])

# Mantener solo columnas necesarias para backend + 'retraso' para entrenamiento
df = df[['aerolinea', 'origen', 'destino', 'retraso', 'distancia_km', 'fecha_partida', 'dia_semana', 'hora_salida', 'Time']] #NS: Mantenemos Time para entrenar el modelo.

# Verificación rápida
print(df.head())
print(df.dtypes)

  aerolinea origen destino  retraso  distancia_km        fecha_partida  \
0        CO    SFO     IAH        1   2733.333252  2018-12-07T00:15:00   
1        US    PHX     CLT        1   2960.000000  2018-12-20T00:15:00   
2        AA    LAX     DFW        1   2200.000000  2018-12-29T00:20:00   
3        AA    SFO     DFW        1   2600.000000  2018-12-15T00:20:00   
4        AS    ANC     SEA        0   2693.333252  2018-12-11T00:30:00   

   dia_semana hora_salida  Time  
0           3    00:15:00    15  
1           3    00:15:00    15  
2           3    00:20:00    20  
3           3    00:20:00    20  
4           3    00:30:00    30  
aerolinea         object
origen            object
destino           object
retraso            uint8
distancia_km     float32
fecha_partida     object
dia_semana          int8
hora_salida       object
Time               int64
dtype: object


In [ ]:
# -----------------------------------------------------
# Paso 3: One-Hot-Encoding de variables categóricas
# -----------------------------------------------------

from sklearn.preprocessing import OneHotEncoder

categorical_features = ['aerolinea', 'origen', 'destino', 'dia_semana']      # NS: Día semana es una variable categorica, no debe ser escalada, si no aplicar OHE, ya que cada numero representa un día no un valor numerico.
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat = ohe.fit_transform(df[categorical_features])
X_cat = pd.DataFrame(X_cat, columns=ohe.get_feature_names_out(categorical_features), index=df.index)

In [ ]:
# -----------------------------------------------------
# CREANDO UN DATAFRAME PARA ENTRENAMIENTO
# El dataframe para backend se limpia de columnas
# innecesarias
# -----------------------------------------------------


# Crear un dataframe para entrenamiento con toda la información necesaria
df_entrenamiento = df.copy()  # incluye columnas auxiliares como 'retraso', 'dia_semana', 'hora_salida'

# Dejar en df original solo las columnas que pide el contrato backend
df = df[['aerolinea', 'origen', 'destino', 'retraso', 'distancia_km', 'hora_salida', 'dia_semana', 'Time']]

#============================================================================================================================
#NS: No es que contrato BackEnd requiera un DataFrame distinto, es que el contrato indica para nosotros en Ciencia de Datos,
# cuales seran las columnas para entrenar el modelo, por lo tanto debe mantenerse las mismas columnas en todo el proceso.
#============================================================================================================================

# Verificación
print("Columnas df_entrenamiento (entrenamiento interno):")
print(df_entrenamiento.head())

print("\nColumnas df (solo contrato backend):")
print(df.head())

Columnas df_entrenamiento (entrenamiento interno):
  aerolinea origen destino  retraso  distancia_km        fecha_partida  \
0        CO    SFO     IAH        1   2733.333252  2018-12-07T00:15:00   
1        US    PHX     CLT        1   2960.000000  2018-12-20T00:15:00   
2        AA    LAX     DFW        1   2200.000000  2018-12-29T00:20:00   
3        AA    SFO     DFW        1   2600.000000  2018-12-15T00:20:00   
4        AS    ANC     SEA        0   2693.333252  2018-12-11T00:30:00   

   dia_semana hora_salida  Time  
0           3    00:15:00    15  
1           3    00:15:00    15  
2           3    00:20:00    20  
3           3    00:20:00    20  
4           3    00:30:00    30  

Columnas df (solo contrato backend):
  aerolinea origen destino  retraso  distancia_km hora_salida  dia_semana  \
0        CO    SFO     IAH        1   2733.333252    00:15:00           3   
1        US    PHX     CLT        1   2960.000000    00:15:00           3   
2        AA    LAX     DFW     

In [ ]:
# -----------------------------------------------------
# Paso 4: Escalar la variable de distancia (si modelo
# lo requiere)
# -----------------------------
# NOTA IMPORTANTE
# 'hora_salida' y 'dia_semana' se usan únicamente como features de entrenamiento.                 NS: Ambas Features, si son importantes para el contrato, y para el modelo final
# No forman parte del contrato backend y deben eliminarse del dataset final                           Por lo tanto deben mantenerse en el modelo. Recordar que nosotros no entregamos
# antes de enviar o guardar los datos para producción.                                                un dataset final, entregamos un modelo, y en producción se necesita saber
# -----------------------------                                                                       exactamente que columnas se usaron en el entrenamiento.


from sklearn.preprocessing import StandardScaler
from scipy import sparse

# Solo las variables numéricas necesarias para el contrato
numeric_features = ['distancia_km']                                                              #NS: La única feature numerica es la distancia_km, pues la hora tendrá un preprocesamiento distinto
X_num = df_entrenamiento[numeric_features].astype('float32')                                     #    y el día semana es una variable que indica un día, no un valor numerico.

# One-Hot encoding ya creado en la Parte 3
# X_cat es la matriz de categorías codificadas

# Convertir variables numéricas a sparse
X_num_sparse = sparse.csr_matrix(X_num.values)

# Escalador (solo para Logistic Regression)
scaler = StandardScaler(with_mean=False)
X_num_scaled = scaler.fit_transform(X_num_sparse)  # ahora solo hay 1 columna

# -----------------------------
# PREPROCESAMIENTO DE LA HORA
# USANDO Time (minutos desde medianoche)
# Este fragmento de codigo fue insertado por Nicolás Staffelbach
# -----------------------------
# Time ∈ [0, 1439]
hora_frac = df_entrenamiento["Time"].astype("float32") / 1440.0

hora_sin = np.sin(2 * np.pi * hora_frac).astype("float32")
hora_cos = np.cos(2 * np.pi * hora_frac).astype("float32")

# Convertir a sparse (NO se escalan)
X_hora = np.column_stack([hora_sin, hora_cos])
X_hora_sparse = sparse.csr_matrix(X_hora)

# Concatenar con variables categóricas One-Hot
from scipy.sparse import hstack
X_logreg_sparse = hstack([X_num_scaled, X_hora_sparse, sparse.csr_matrix(X_cat.values)])    #NS: Se agregó la columna X_hora_sparse, para el entrenamiento


# Variable objetivo para entrenamiento
y = df_entrenamiento['retraso']  # solo para entrenamiento

In [ ]:
# -----------------------------------------------------
# Paso 5: Guardar objetos de transformación para producción
# -----------------------------------------------------

import joblib

# Guardar objetos de transformación
joblib.dump(ohe, 'onehot_encoder.pkl')       # OneHotEncoder de variables categóricas
joblib.dump(scaler, 'scaler_logreg.pkl')    # Escalador de distancia solo para Logistic Regression

print("Objetos de transformación guardados para producción")

Objetos de transformación guardados para producción


## Machine Learning Engineer (MLE)

Sección para las tareas de Machine Learning Engineer

Encargado: Luis Jácome

### Entrenamiento y Evaluación base

##Split de Datos (Train/Test) con un random_state fijo
En esta primera tarea separaremos los datos en entrenamiento y prueba con el objetivo de que sea reproducible, se encuentre balanceada y lista para entrenar al modelo al cual se le asignará el nombre de champion.

In [ ]:
from sklearn.model_selection import train_test_split

# --------------------------------------------------
# SPLIT TRAIN / TEST (MLE - Tarea 1)
# --------------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X_logreg_sparse,
    y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

# Verificación rápida
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

print("\nDistribución Delay (train):")
print(y_train.value_counts(normalize=True))

print("\nDistribución Delay (test):")
print(y_test.value_counts(normalize=True))

Train shape: (431506, 614)
Test shape: (107877, 614)

Distribución Delay (train):
retraso
0    0.554558
1    0.445442
Name: proportion, dtype: float64

Distribución Delay (test):
retraso
0    0.554558
1    0.445442
Name: proportion, dtype: float64


## Entrenar el modelo seleccionado con parametros por defecto
Entrenaremos un modelo baseline usando los datos ya escalados y dejar el modelo listo para inferencia.
Se utilizará Logistic Regression para el entrenamiento ya que es un modelo interpretable, rápido, robusto y adecuado como baselinepara clasificación binaria.

In [ ]:
from sklearn.linear_model import LogisticRegression
import joblib

# --------------------------------------------------
# ENTRENAMIENTO MODELO BASE - LOGISTIC REGRESSION
# --------------------------------------------------

champion = LogisticRegression(
    random_state=42,
    max_iter=1000,
    solver='liblinear'
)

champion.fit(X_train, y_train)

print("Modelo Logistic Regression entrenado correctamente")

# --------------------------------------------------
# GUARDAR MODELO PARA PRODUCCIÓN
# --------------------------------------------------

joblib.dump(champion, 'champion.pkl')

print("Modelo guardado como champion.pkl")


Modelo Logistic Regression entrenado correctamente
Modelo guardado como champion.pkl


## Evaluación del modelo
evaluaremos el modelo baseline mediante métricas estándar de clasificación binaria (Accuracy, Precision, Recall y F1-score) tanto en el conjunto de entrenamiento como de prueba, utilizando un umbral de decisión por defecto de 0.5

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# --------------------------------------------------
# PREDICCIONES
# --------------------------------------------------
y_train_pred = champion.predict(X_train)
y_test_pred = champion.predict(X_test)
# --------------------------------------------------
# MÉTRICAS TRAIN
# --------------------------------------------------
train_metrics = {
    "Accuracy": accuracy_score(y_train, y_train_pred),
    "Precision": precision_score(y_train, y_train_pred),
    "Recall": recall_score(y_train, y_train_pred),
    "F1-Score": f1_score(y_train, y_train_pred)
}
# --------------------------------------------------
# MÉTRICAS TEST
# --------------------------------------------------
test_metrics = {
    "Accuracy": accuracy_score(y_test, y_test_pred),
    "Precision": precision_score(y_test, y_test_pred),
    "Recall": recall_score(y_test, y_test_pred),
    "F1-Score": f1_score(y_test, y_test_pred)
}
# --------------------------------------------------
# MOSTRAR RESULTADOS
# --------------------------------------------------
print("Métricas Train:")
for k, v in train_metrics.items():
    print(f"{k}: {v:.4f}")

print("\nMétricas Test:")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

Métricas Train:
Accuracy: 0.6500
Precision: 0.6357
Recall: 0.5016
F1-Score: 0.5608

Métricas Test:
Accuracy: 0.6502
Precision: 0.6363
Recall: 0.5012
F1-Score: 0.5607


## Validar que el modelo no tenga overfitting excesivo
La ausencia de overfitting se evidencia en la similitud casi exacta entre las métricas de entrenamiento y prueba. Las diferencias inferiores al 0.1% indican que el modelo generaliza correctamente y no presenta alta varianza.

## Seleccionar el umbral de probabilidad óptimo para la clasificación


In [ ]:
# Probabilidad de clase positiva (Delay = 1)
y_proba_test = champion.predict_proba(X_test)[:, 1]

In [ ]:
# Rango de umbrales a evaluar
thresholds = np.arange(0.1, 0.9, 0.05)

results = []

for threshold in thresholds:
    y_pred_threshold = (y_proba_test >= threshold).astype(int)

    precision = precision_score(y_test, y_pred_threshold)
    recall = recall_score(y_test, y_pred_threshold)
    f1 = f1_score(y_test, y_pred_threshold)

    results.append({
        "Threshold": threshold,
        "Precision": precision,
        "Recall": recall,
        "F1_score": f1
    })

# Resultados en DataFrame
threshold_df = pd.DataFrame(results)

threshold_df


,Threshold,Precision,Recall,F1_score
0,0.10,0.446242,0.999584,0.617026
1,0.15,0.451505,0.994340,0.621020
2,0.20,0.464573,0.975340,0.629367
3,0.25,0.483294,0.938256,0.637970
4,0.30,0.504759,0.879550,0.641419
5,0.35,0.531903,0.805132,0.640599
6,0.40,0.561833,0.710694,0.627557
7,0.45,0.597232,0.607995,0.602566
8,0.50,0.636282,0.501155,0.560692
9,0.55,0.679885,0.404241,0.507021


In [ ]:
# selección del umbral óptimo (máximo F1)
best_threshold_row = threshold_df.loc[threshold_df['F1_score'].idxmax()]
best_threshold_row

,4
Threshold,0.300000
Precision,0.504759
Recall,0.879550
F1_score,0.641419


In [ ]:
# guardado del umbral óptimo
best_threshold = best_threshold_row['Threshold']

print(f"Umbral óptimo seleccionado: {best_threshold:.2f}")


Umbral óptimo seleccionado: 0.30


In [ ]:
# Comparación directa vs umbral 0.5
# Predicción con umbral por defecto
y_pred_default = (y_proba_test >= 0.5).astype(int)

# Predicción con umbral óptimo
y_pred_optimal = (y_proba_test >= best_threshold).astype(int)

comparison = pd.DataFrame({
    "Metric": ["Precision", "Recall", "F1_score"],
    "Threshold_0.5": [
        precision_score(y_test, y_pred_default),
        recall_score(y_test, y_pred_default),
        f1_score(y_test, y_pred_default)
    ],
    "Optimal_Threshold": [
        precision_score(y_test, y_pred_optimal),
        recall_score(y_test, y_pred_optimal),
        f1_score(y_test, y_pred_optimal)
    ]
})

comparison


,Metric,Threshold_0.5,Optimal_Threshold
0,Precision,0.636282,0.504759
1,Recall,0.501155,0.879550
2,F1_score,0.560692,0.641419


Al evaluar distintos umbrales de decisión, se identificó un umbral óptimo que maximiza el F1-score el cual es 0.3. Este ajuste incrementa significativamente la capacidad del modelo para detectar vuelos retrasados (Recall ≈ 88%), a costa de una reducción moderada en Precision, logrando un mejor equilibrio general sin necesidad de reentrenar el modelo.

## Machine Learning Operations (MLOps)

Sección para las tareas de Machine Learning Operations

Encargado: Nicolás Staffelbach

### Microservicio Python

#### Validación de las versiones de las librerias en Colab


```
!pip show fastapi...
```
Este código tiene como fin el saber las versiones de las librerias utilizadas en el entorno Google Colab, para la creación del archivo `requirements.txt` para garantizar el funcionamiento del modelo en producción.


In [ ]:
#!pip show fastapi scikit-learn pandas numpy joblib uvicorn pydantic

#### Script de carga del modelo

En esta sección se desarrolla la creación del pipeline de carga del encoder, scaler y modelo, para su posterior uso en producción, garantizando el uso de los mismos objetos utilizados para el entrenamiento del modelo. Y también garantizando la optimización de la API.

In [ ]:
%%writefile inference_pipeline.py
import joblib
import pandas as pd
import numpy as np
from scipy import sparse

# -----------------------------
# CARGA DE ARTEFACTOS
# -----------------------------
model = joblib.load("champion.pkl")
ohe = joblib.load("onehot_encoder.pkl")
scaler = joblib.load("scaler_logreg.pkl")

# -----------------------------
# DEFINICIÓN DE FEATURES
# -----------------------------
# dia_semana VA COMO CATEGÓRICA
CATEGORICAL_FEATURES = ["aerolinea", "origen", "destino", "dia_semana"]

# SOLO las numéricas escaladas
NUMERIC_FEATURES = ["distancia_km"]

# Variables cíclicas (NO se escalan)
CYCLIC_FEATURES = ["hora_sin", "hora_cos"]

# -----------------------------
# PREPROCESAMIENTO
# -----------------------------
def preprocess(payload: dict):
    df = pd.DataFrame([payload])

    # -------------------------
    # PARSE DATETIME
    # -------------------------
    dt = pd.to_datetime(df["fecha_partida"])

    # Día de la semana (0=lunes, 6=domingo)
    df["dia_semana"] = dt.dt.dayofweek.astype("int8")

    # -------------------------
    # HORA CÍCLICA
    # -------------------------
    # Hora fraccional
    hora_frac = (dt.dt.hour + dt.dt.minute / 60.0) / 24.0

    df["hora_sin"] = np.sin(2 * np.pi * hora_frac).astype("float32")
    df["hora_cos"] = np.cos(2 * np.pi * hora_frac).astype("float32")

    # -------------------------
    # CATEGÓRICAS (One-Hot)
    # -------------------------
    X_cat = ohe.transform(df[CATEGORICAL_FEATURES])
    X_cat = sparse.csr_matrix(X_cat)

    # -------------------------
    # NUMÉRICAS (ESCALADAS)
    # -------------------------
    X_num = df[NUMERIC_FEATURES].astype("float32")
    X_num_sparse = sparse.csr_matrix(X_num.values)
    X_num_scaled = scaler.transform(X_num_sparse)

    # -------------------------
    # CÍCLICAS (NO ESCALADAS)
    # -------------------------
    X_cyc = df[CYCLIC_FEATURES].astype("float32")
    X_cyc = sparse.csr_matrix(X_cyc.values)

    # -------------------------
    # CONCATENACIÓN FINAL
    # ORDEN CRÍTICO
    # -------------------------
    X = sparse.hstack([
        X_num_scaled,  # distancia_km
        X_cyc,         # hora_sin, hora_cos
        X_cat          # categóricas (incluye dia_semana)
    ])

    return X

# -----------------------------
# PREDICCIÓN
# -----------------------------
def predict(payload: dict):
    X = preprocess(payload)
    proba = model.predict_proba(X)[0, 1]

    prediction = "Retrasado" if proba >= 0.3 else "No Retrasado"

    return {
        "prevision": prediction,
        "probabilidad": round(float(proba), 2)
    }

Writing inference_pipeline.py


#### Creación del microservicio

A través del Notebook se crea un archivo `app.py` para la implementación del microservicio a través de la libreria `fastapi`

In [ ]:
%%writefile app.py
from fastapi import FastAPI
from pydantic import BaseModel, Field
from inference_pipeline import predict

app = FastAPI(title="Flight Delay Predictor")

class PredictionInput(BaseModel):
    aerolinea: str = Field(..., example="AZ")
    origen: str = Field(..., example="GIG")
    destino: str = Field(..., example="GRU")
    fecha_partida: str = Field(..., example="2025-11-10T14:30:00")
    distancia_km: float = Field(..., gt=0)

class PredictionOutput(BaseModel):
    prevision: str
    probabilidad: float

@app.post("/predict", response_model=PredictionOutput)
def predict_delay(data: PredictionInput):
    return predict(data.dict())

Writing app.py


#### Prueba del Microservicio

Ahora se inicia el proceso de testeo del Microservicio para validar que se cumple el contrato

In [ ]:
from fastapi.testclient import TestClient
from app import app
import time

client = TestClient(app)

payload = {
    "aerolinea": "AZ",
    "origen": "GIG",
    "destino": "GRU",
    "fecha_partida": "2025-11-10T14:30:00",
    "distancia_km": 350
}

start = time.perf_counter()
response = client.post("/predict", json=payload)
latency_ms = (time.perf_counter() - start) * 1000

print("Status code:", response.status_code)
print("Response JSON:", response.json())
print(f"Latency: {latency_ms:.2f} ms")

Status code: 200
Response JSON: {'prevision': 'Retrasado', 'probabilidad': 0.38}
Latency: 72.89 ms


## Data Analyst (DA)

Sección para las tareas de Data Analyst

Encargado: David Aragón

### Análisis de Datos Exploratorio EDA

### Análisis Exploratorio de Datos (EDA)
Realizado por: David Aragón (DA)

Objetivo: Explorar el dataset para identificar patrones, distribuciones y correlaciones básicas entre variables como aerolínea, origen/destino, día de la semana, hora, duración y retraso. Usamos Pandas para summaries, Matplotlib/Seaborn para visualizaciones, y correlaciones para detectar relaciones (ej: ¿los retrasos correlacionan con la hora o distancia?).

Dataset: Airlines.csv (539k registros, 9 columnas). No hay nulos evidentes de df.info().

In [ ]:
# Importar librerías necesarias (agrega si no están)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# **[INSERT: DOC HERE]** - Documentación: Función para EDA básico.
def perform_eda(df):
    # Resumen estadístico
    print("Resumen estadístico:")
    print(df.describe())

    # Distribución de retrasos (target: Delay)
    print("\nDistribución de Delay (0: Puntual, 1: Retrasado):")
    print(df['Delay'].value_counts(normalize=True) * 100)  # Porcentajes

    # Visualización: Distribución de Delay
    sns.countplot(x='Delay', data=df)
    plt.title('Distribución de Retrasos en Vuelos')
    plt.show()

    # Correlaciones básicas (solo numéricas)
    corr_matrix = df.select_dtypes(include=['int64', 'float64']).corr()
    print("\nMatriz de Correlaciones:")
    print(corr_matrix['Delay'].sort_values(ascending=False))  # Correlación con Delay

    # Heatmap de correlaciones
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.title('Mapa de Calor de Correlaciones')
    plt.show()

    # Análisis por categóricas (ej: retrasos por aerolínea)
    print("\nRetrasos por Aerolínea:")
    print(df.groupby('Airline')['Delay'].mean().sort_values(ascending=False))

    # Visualización: Retrasos por Día de la Semana
    sns.barplot(x='DayOfWeek', y='Delay', data=df, estimator=np.mean)
    plt.title('Tasa de Retrasos por Día de la Semana')
    plt.ylabel('Tasa de Retraso')
    plt.show()

# Ejecutar EDA
perform_eda(df)  # df ya cargado en sección DE

Resumen estadístico:
             retraso   distancia_km     dia_semana           Time
count  539383.000000  539383.000000  539383.000000  539383.000000
mean        0.445442    1762.693237       3.929668     802.728963
std         0.497015     935.015320       1.914664     278.045911
min         0.000000       0.000000       1.000000      10.000000
25%         0.000000    1080.000000       2.000000     565.000000
50%         0.000000    1533.333374       4.000000     795.000000
75%         1.000000    2160.000000       5.000000    1035.000000
max         1.000000    8733.333008       7.000000    1439.000000

Distribución de Delay (0: Puntual, 1: Retrasado):


KeyError: 'Delay'